# cls21 Analysis Notebook

In [ ]:
%matplotlib widget

## Path configuration

In [ ]:
import sys
sys.path.append('/path/to/pythib/')

## Data Model
The data model created here is the central object that will hold all analysis results. Some information about each ensemble is stored in the data file,

- lattice spacing from Bruno, Korzec, Schaefer [[arXiv:1608.08900]](http://arxiv.org/abs/arXiv:1608.08900)
- $\hat Z_V$ and $c_V$ from Gerardin, Harris, Meyer [in preparation]

In [ ]:
import widget_ana

ensName = 'd200'

dM = widget_ana.AnalysisModel('dat_final/cls21_'+ensName+'.hdf5', 'dat_final/cls21_'+ensName+'_analysis.hdf5')

print(dict(dM.latParDict))

## Single hadrons

* start the widget in the cell below
* add the analysis with $t_\mathrm{max} = 30$, which is the default value
* the $t_\mathrm{min}$ plot is being produced in a background process -- click on the button in the analysis list on the left to view once finished
* use standard matplotlib interaction to zoom etc.
* clicking on a point in the $t_\mathrm{min}$ plot selects this energy -- clicking again unselects
* once a value for each level has been chosen, save the data and proceed

In [ ]:
aW = widget_ana.AnalysisWidget(dM, 'sh')
aW.showWidget()

## Pion dispersion relation

In [ ]:
from backend_ana import errorBootstrap

import matplotlib.pyplot as plt
import numpy as np

dM._fetchResults()

nLev = len(dM.anaDict['pion']['mSmpls'])
dispRel = np.array([np.array([aLev[0]**2,]+[anErr for anErr in errorBootstrap(aLev**2)]) for aLev in dM.anaDict['pion']['mSmpls']])

fig = plt.figure()
ax = fig.add_subplot(111)
ax.errorbar(x=range(nLev), y=dispRel[:,0], yerr=dispRel[:,1:3].T, fmt='k+', markersize=2)
ax.plot([0, nLev-1], [dispRel[0,0], dispRel[0,0]+(2.*np.pi/dM.latParDict['spatExt'])**2*(nLev-1)], color='0.7', ls='-')
fig.show()


## Pion-pion spectrum

In [ ]:
aW = widget_ana.AnalysisWidget(dM, 'nonsh')
aW.showWidget()

## Output: Save $t_{min}$ plots

In [ ]:
#aW.axList[0].set_xlabel('$t_\mathrm{min}$', fontsize=20)
#aW.axList[0].set_ylabel('$E$', fontsize=20)
#aW.axList[0].tick_params(axis='both', which='major', labelsize=16)
#aW.axList[0].set_ylim([0.274,0.298])

aW.fig.savefig('plot_n200/pSq4-Ep.pdf', transparent=True)

## Compute box matrix elements
If this cell fails with a KeyError about 'mSmpls', allow some more time for fit result calculation.

In [ ]:
import BMat

import numpy as np

    # for the progress bar
import ipywidgets as widgets
from IPython.display import display

dM._fetchResults()

    ####################
    #
    #   amplitude parametrization
    #
    ####################

    # take ell=1 into account
def isZero(JtimesTwo, Lp, SptimesTwo, chanp, L, StimesTwo, chan):
    return not (JtimesTwo==2
            and Lp==1 and L==1
            and chanp==0 and chan==0
            and SptimesTwo==0 and StimesTwo==0)

    #
    # ell=1: BW amplitude for \tilde K^-1(E_cm) as given in (2.9)
    # of 1802.03100
class BWAmplitude:
    def __init__(self, _mr, _g):
        self.mr = _mr
        self.g = _g
        
    def bwFunc(self, Ecm_over_mref):
        return (self.mr**2 - Ecm_over_mref**2) * 6.*np.pi * Ecm_over_mref / (self.g**2)

    def calcFunc(self, JtimesTwo, Lp, SptimesTwo, chanp, L, StimesTwo, chan, Ecm_over_mref, pSqFuncList):
        return self.bwFunc(Ecm_over_mref)
    
        # below we're assuming that mpi = 1, i.e. that mpi is used as the reference energy
    def pcm(self, Ecm_over_mref):
        return (0.25*Ecm_over_mref**2 - 1.0)**0.5

    def dEstardq(self, Ecm_over_mref):
        return 4.*self.pcm(Ecm_over_mref) / Ecm_over_mref

    def qTimesdeltaDeriv(self, Ecm_over_mref):
        return 1./(self.pcm(Ecm_over_mref)**3 * (1.+self.bwFunc(Ecm_over_mref)**2/self.pcm(Ecm_over_mref)**6)) * \
                (3.* self.bwFunc(Ecm_over_mref) + 6.*np.pi*self.pcm(Ecm_over_mref)/self.g**2 * (3.*Ecm_over_mref**2 - self.mr**2) * self.dEstardq(Ecm_over_mref) )
        
    def phaseParametrization(self, x):
        phaseData = np.arctan(np.divide(self.pcm(x)**3,self.bwFunc(x)))
        phaseData = phaseData + (phaseData < 0.0).astype(int)*np.pi
        return phaseData


    # create a KMatrix Object
bwAmpl = BWAmplitude(None, None)
Kinv = BMat.KMatrix(bwAmpl.calcFunc, isZero)


    # name1, name2, spin1, spin2, identicalParticles, sameIntrinsicParities
chanList = [BMat.DecayChannelInfo('pion','pion',0,0,True,True),]

allLevelList = []
momRay = {0 : 'ar', 1 : 'oa', 2 : 'pd', 3 : 'cd', 4 : 'oa'}

    # need SH masses
mpiSmpls = dM.anaDict['pion']['mSmpls'][0]
mpiLSmpls = mpiSmpls * dM.latParDict['spatExt']

    # discard inelastic levels based on the mean
def isElastic(aLev):
    return (aLev['mSmpls'][0]**2 - (2. * np.pi / dM.latParDict['spatExt'])**2 * aLev['irrepKey'][0])**0.5 / mpiSmpls[0] < dM.latParDict['inelEcm']

def isEll1Dominated(aLev):
    return aLev['irrepKey'] in [(0,'T1up'),(1,'A1p'),(1,'Ep'),(2,'A1p'),(2,'B1p'),(2,'B2p'),(3,'A1p'),(3,'Ep'),(4,'A1p'),(4,'Ep')]

for chanKey in dM.getNonSHList():
    momSq = int(chanKey[3])
    irName = chanKey.split('-')[1]

    # create all FastBQ instances
    allLevelList.extend([{
        'irrepKey'  :   (momSq, irName),
        'mSmpls'    :   someSamples,
        'boxQuant'  :   BMat.FastBoxQuantization(momRay[momSq], momSq, irName[:-1], chanList, [1,], Kinv, True),
        } for someSamples in dM.anaDict[chanKey]['mSmpls'] if (not (someSamples is None)) and isEll1Dominated({'irrepKey' : (momSq, irName)})])

    # list of SH masses for each decay channel
shMassList = [(np.ones_like(mpiSmpls), np.ones_like(mpiSmpls))]

levelList = [aLev for aLev in allLevelList if isElastic(aLev) and isEll1Dominated(aLev)]
nLev = len(levelList)

    # initialize
    
wInitProgBar = widgets.IntProgress(min=0, max=nLev-1, description='Initializing')
display(wInitProgBar)

for iLev, aLev in enumerate(levelList):
    wInitProgBar.value = iLev
    aLev['boxQuant'].initialize(aLev['mSmpls']/mpiSmpls, mpiLSmpls, shMassList)

    # /initializing

    # produce plot
    
resList = []

for aLev in levelList:
    pcotSmpls = np.array(aLev['boxQuant'].getBoxMatrixElementList(0,0)).real
    pcotErr = errorBootstrap(pcotSmpls)
        
    mVal = aLev['boxQuant'].getEcmOverMrefList()[0]
    mErr = errorBootstrap(np.array(aLev['boxQuant'].getEcmOverMrefList()))
    
    print(aLev['irrepKey'][0], aLev['irrepKey'][1], '\t', mVal, mErr[0], mErr[1], pcotSmpls[0], pcotErr[0], pcotErr[1])
    resList.append(np.array([mVal, mErr[0], mErr[1], pcotSmpls[0], pcotErr[0], pcotErr[1]]))

resA = np.array(resList)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.errorbar(x=resA[:,0], xerr=resA[:,1:3].T, y=resA[:,3], yerr=resA[:,4:6].T, fmt='k.')

fig.show()

## Breit-Wigner fit

In [ ]:
from numpy.linalg import cholesky, norm
from scipy.linalg import solve_triangular
from scipy.optimize import minimize


    
nLev = len(levelList)
print('number of levels =', nLev)

nSmpls = mpiSmpls.shape[0]
nDof = nLev - 2

    ####################
    #
    #   correlated chi^2
    #
    ####################

def covBootstrap(theData):
    diffs = theData[1:] - theData[0]
    return 1./float(nSmpls-2) * np.tensordot(diffs.conj(), diffs, axes=(0,0)).real


def configSelector(iCfg):

        # len(fitPar) == 2 is not checked explicitly
    def chiSq(fitPar):

        muVal = 10.

        bwAmpl.mr = fitPar[0]
        bwAmpl.g = fitPar[1]

        # compute determinant residual for each level
        #   index order (nSampls, nLev)

        theDat = np.array([aLev['boxQuant'].getOmegaList(muVal) for aLev in levelList]).T

        # compute covariance matrix from bootstrap samples
        covMat = covBootstrap(theDat)
        cholCov = cholesky(covMat)
        residue = theDat[iCfg]

        return norm(solve_triangular(cholCov, residue, lower=True, check_finite=False))**2 / nDof

    return chiSq


    ####################
    #
    #   Perform a minimization
    #
    ####################
    
wProgBar = widgets.IntProgress(min=0, max=nSmpls-1, description='Running fit')
display(wProgBar)

mrList = []
gList = []

for iCfg in range(nSmpls):
    wProgBar.value = iCfg
    chiSqFunc = configSelector(iCfg)

    initGuess = [mrList[0], gList[0]] if iCfg > 0 else [3., 6.0]
    minObj = minimize(chiSqFunc, initGuess)

    mrList.append(minObj.x[0])
    gList.append(minObj.x[1])
    
    if (iCfg == 0):
        print('mean:',[mrList[0], gList[0]],'chiSq/dof =',minObj.fun)

mrErr = errorBootstrap(np.array(mrList))
gErr = errorBootstrap(np.array(gList))

print('mr =', mrList[0], mrErr[0], mrErr[1])
print('g =', gList[0], gErr[0], gErr[1])

## Debug: Check validity of minimum on the mean (requires iminuit)
If you don't trust the scipy optimization parameters, you can check the minimum with Minuit.

In [ ]:
from iminuit import Minuit

chiSqFunc = configSelector(0)

def wrapFunc(mr, g): return chiSqFunc([mr, g])

m = Minuit(wrapFunc, mr=3.3, g=6.0)
m.migrad()
print(m.values)

## Current matrix elements

In [ ]:
aW = widget_ana.CurrentWidget(dM)
aW.showWidget()

## OA/CU Plot

In [ ]:
from backend_ana import errorBootstrap

import ipympl
import matplotlib.pyplot as plt
import numpy as np

dM._fetchResults()

mpiSmpls = dM.anaDict['pion']['mSmpls'][0]

resList = []

    # while we're here, let's try to put cuSmpls and oaSmpls into levelList into the right place
iLev = 0

for chanKey in dM.getNonSHList():
    momSq = int(chanKey[3])
    irName = chanKey.split('-')[1]
    
    for mSmpls, cuSmpls, oaSmpls in zip(dM.anaDict[chanKey]['mSmpls'], dM.anaDict['CU'+chanKey]['ovSmpls'], dM.anaDict['OA'+chanKey]['ovSmpls']):
            # just bookkeeping to get cu and oa samples lined up in levelList
        if mSmpls is None or not isElastic({'irrepKey' : (momSq, irName), 'mSmpls' : mSmpls}): continue
        if cuSmpls is None or oaSmpls is None:
            iLev = iLev + 1
            continue
            
        if momSq != levelList[iLev]['irrepKey'][0] or irName != levelList[iLev]['irrepKey'][1]:
            print('Warning: Mismatch in levelList for cu/oa/mSmpls:',iLev,levelList[iLev]['irrepKey'],momSq,irName)
            
        levelList[iLev]['cuSmpls'] = cuSmpls
        levelList[iLev]['oaSmpls'] = oaSmpls
        iLev = iLev + 1
            # /done -- now for the plot data
            
        EcmSmpls = (mSmpls**2 - (2. * np.pi / dM.latParDict['spatExt'])**2 * momSq)**0.5
        EcmErr = errorBootstrap(EcmSmpls)
        
        oacuSmpls = oaSmpls/cuSmpls
        oacuErr = errorBootstrap(oacuSmpls)
        
        resList.append(np.array([EcmSmpls[0], EcmErr[0], EcmErr[1], oacuSmpls[0], oacuErr[0], oacuErr[1]]))
        

resA = np.array(resList)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.errorbar(x=resA[:,0], xerr=resA[:,1:3].T, y=resA[:,3], yerr=resA[:,4:6].T, fmt='k.')

ax.set_xlabel('$a E_{cm}$')
ax.set_ylabel('OA/CU')
fig.show()

## LLM factors

The LLM factor for irrep $\Lambda$ is given by

$$g^\Lambda(\gamma) \left ( q \frac{d\delta_1}{dq} + q \frac{d\phi^\Lambda}{dq} \right) \frac{3 \pi E^{*2}}{2q^5 L^3} \qquad \mathrm{with} \; g^\Lambda = \begin{cases}\gamma^{-1} & \Lambda = A_1^+ \\ \gamma & \mathrm{otherwise}\end{cases}$$

and we compute the pieces as follows

$$q \frac{d \, \cdot}{dq} = -\frac{q}{1+\cot^2 \cdot} \frac{d \cot \cdot}{dE^*} \frac{d E^*}{dq}$$

In [ ]:
derStep = 1.0e-4

    # slight abuse of BWAmplitude here: _mr and _g can be np.arrays, so that
    # bwAmpl is fully vectorized
def deltaDerivWrapper(_mr, _g, _Ecm_over_mref):
    bwAmpl.mr = np.array(_mr)
    bwAmpl.g = np.array(_g)
    return bwAmpl.qTimesdeltaDeriv(_Ecm_over_mref)


for aLev in levelList:
    gammaSmpls = np.array(aLev['boxQuant'].getElabOverMrefList()) / np.array(aLev['boxQuant'].getEcmOverMrefList())
    
        # create an old-fashioned BoxQuantization object to get q^2 samples and u d\phi/du
    BQ = BMat.BoxQuantization(momRay[aLev['irrepKey'][0]], aLev['irrepKey'][0], aLev['irrepKey'][1][:-1], chanList, [1,], Kinv, True)
    BQ.setMassesOverRef(0, 1.0, 1.0)
    qSqFunc = BQ.getQcmSqFunctions()[0]
    
    nSmpls = aLev['mSmpls'].shape[0]
    qSqSmpls = []
    derCotPhiEcmSmpls = []
    
    for aMpiL, aEcm in zip(mpiLSmpls, aLev['boxQuant'].getEcmOverMrefList()):
        BQ.setRefMassL(aMpiL)
        
        qSqSmpls.append(qSqFunc(aEcm))

        cotPhiUp = BQ.getBoxMatrixFromEcm(aEcm + derStep).real / qSqFunc(aEcm + derStep)**1.5
        cotPhiLow = BQ.getBoxMatrixFromEcm(aEcm - derStep).real / qSqFunc(aEcm - derStep)**1.5
        
        derCotPhiEcmSmpls.append(0.5*(cotPhiUp - cotPhiLow) / derStep)
        
    qSqSmpls = np.array(qSqSmpls)
    
    uDerPhi = 1./(1. + np.array(aLev['boxQuant'].getBoxMatrixElementList(0,0)).real**2/qSqSmpls**3) * np.array(derCotPhiEcmSmpls) * \
                4. * qSqSmpls / np.array(aLev['boxQuant'].getEcmOverMrefList())
    qDerDelta = deltaDerivWrapper(mrList, gList, np.array(aLev['boxQuant'].getEcmOverMrefList()))
    
    gFac = 1./gammaSmpls if aLev['irrepKey'][1] == 'A1p' else gammaSmpls
    aLev['llmFac'] = gFac * (qDerDelta + uDerPhi) * 1.5 * np.pi * np.array(aLev['boxQuant'].getEcmOverMrefList())**2 / (qSqSmpls**2.5*mpiLSmpls**3)
    
    print(aLev['irrepKey'][0], aLev['irrepKey'][1], gammaSmpls[0], aLev['llmFac'][0], qSqSmpls[0], aLev['boxQuant'].getEcmOverMrefList()[0], aLev['boxQuant'].getElabOverMrefList()[0])

## Renormalization, O(a) improvement, and form factor with GS parametrization

In [ ]:
class GS:
    
    def setVals(self, _mr, _g):
        self.mr = _mr
        self.g = _g
        self.krho = (0.25*self.mr*self.mr - 1.)**0.5
        self.hmrho = 2./np.pi*self.krho/self.mr*np.log(0.5*(self.mr+2.*self.krho))
        self.hdermrho = 2./np.pi*((0.25/self.krho-self.krho/(self.mr**2))*np.log(0.5*(self.mr+2.*self.krho)) + \
                         self.krho/(self.mr**2)*(1.+2.*self.mr/self.krho)/(1.+2.*self.krho/self.mr))
        self.f0 = (self.hmrho - 1./np.pi) + self.mr**2*(6.*np.pi/(self.g**2) + 0.5*self.krho**2/self.mr*self.hdermrho)
    
    def __init__(self, _mr, _g):
        self.setVals(_mr, _g)
    
    def gsParametrization(self, x):
        return self.f0 /(((self.mr**2-x**2)*(6.0*np.pi)/(self.g**2))**2 + (0.25*x**2 - 1)**3./(x**2))**0.5
    


ZV = dM.latParDict['ZVhat']
cV = dM.latParDict['cV']

resList = []

for aLev in levelList:
    if 'cuSmpls' not in aLev or 'oaSmpls' not in aLev: continue
    EcmOverMpiSmpls = np.array(aLev['boxQuant'].getEcmOverMrefList())
    EcmOverMpiErr = errorBootstrap(EcmOverMpiSmpls)
    
        # we need a sqrt of the llmFac below; throw a warning if we encounter a negative value
    nNeg = (aLev['llmFac']<0.).sum()
    if nNeg > 0:
        print('Warning:', nNeg, 'negative llmFac in', aLev['irrepKey'])
        aLev['llmFac'] = np.fabs(aLev['llmFac'])
    
    nNan = np.isnan(aLev['cuSmpls']).sum()
    if nNan > 0:
        print('Warning:', nNan, 'NaN samples in CU ', aLev['irrepKey'])
        
    nNan = np.isnan(aLev['oaSmpls']).sum()
    if nNan > 0:
        print('Warning:', nNan, 'NaN samples in OA ', aLev['irrepKey'])
    
    
        # 1/sqrt(2) is for isovector part of el.-magn. current
    ffSmpls = 0.5**0.5 * ZV * (aLev['llmFac'] * (aLev['cuSmpls'] + cV * aLev['oaSmpls'])**2)**0.5
    aLev['ffSmpls'] = ffSmpls
    
    ffErr = errorBootstrap(ffSmpls)
    resList.append(np.array([EcmOverMpiSmpls[0], EcmOverMpiErr[0], EcmOverMpiErr[1], ffSmpls[0], ffErr[0], ffErr[1]]))

resA = np.array(resList)

    # GS parametrization data
gs = GS(mrList[0], gList[0])
maxEcm = max(aLev['boxQuant'].getEcmOverMrefList()[0] for aLev in levelList)
fitX = np.linspace(2.0, maxEcm, 200)
fitVal = gs.gsParametrization(fitX)


fig = plt.figure()
ax = fig.add_subplot(111)
ax.errorbar(x=resA[:,0], xerr=resA[:,1:3].T, y=resA[:,3], yerr=resA[:,4:6].T, fmt='k.')
ax.plot(fitX, fitVal, 'k-')

ax.set_xlabel('$E_{cm}/m_\pi$')
ax.set_ylabel('$|F_\pi|$')
fig.show()
    

## Dispersive parametrization of the form factor

### Compute Omnes function

In [ ]:
from scipy.integrate import quad as integrate

nSubtr = 3

def subtrIntegral(deltaFunc, t, nSubtr):
    def integrand(s):
        return (deltaFunc(s**0.5)-deltaFunc(t**0.5))/(s**nSubtr*(s-t))
    
    intVal, intErr = integrate(integrand, 4.0, np.inf)
    return t**nSubtr / np.pi * intVal

def getOmnesFunc(nSubtr):
    
        # twice-subtracted
    def omnesFunc2(deltaFunc, t):
        return np.exp(subtrIntegral(deltaFunc, t, nSubtr) - deltaFunc(t**0.5)/(4.*np.pi)*
                      (t - 8.*np.log(2.) + 4.*np.log(t-4.)))
    
    def omnesFunc3(deltaFunc, t):
        # thrice-subtracted
        return np.exp(subtrIntegral(deltaFunc, t, nSubtr) - deltaFunc(t**0.5)/(32.*np.pi)*
                  (t**2 + 8*t - 64.*np.log(2.) + 32.*np.log(t-4.)))
    
    retDic = {2 : omnesFunc2, 3 : omnesFunc3}
    return retDic[nSubtr]


def dispFF(nSubtr, deltaFunc, polyFunc, t):
    return np.exp(polyFunc(t)) * getOmnesFunc(nSubtr)(deltaFunc, t)

wProgBar = widgets.IntProgress(min=0, max=len(levelList)-1, description='Computing')
display(wProgBar)

    # compute Omnes samples for each level
for iLev, aLev in enumerate(levelList):
    wProgBar.value = iLev
    
    resList = []
    
    for anEcm, aMr, aG in zip(np.array(aLev['boxQuant'].getEcmOverMrefList()), mrList, gList):
        bwAmpl.mr = aMr
        bwAmpl.g = aG
        resList.append(getOmnesFunc(nSubtr)(bwAmpl.phaseParametrization, anEcm**2))
        
    aLev['omnesSmpls'] = np.array(resList)

### Fit polynomial $P_1$

In [ ]:
wProgBar = widgets.IntProgress(min=0, max=nSmpls-1, description='Fitting')
display(wProgBar)

    ####################
    #
    #   correlated chi^2
    #
    ####################

def configSelector(iCfg):
    def chiSq(fitPar):
        
        def polyFunc(s):
            return sum(aPar * s**iPar for iPar, aPar in enumerate(fitPar, start=1))

        # compute residual for each level
        #   index order (nSampls, nLev)
        
        theDat = np.array([np.log(aLev['ffSmpls']/aLev['omnesSmpls']) - polyFunc(np.array(aLev['boxQuant'].getEcmOverMrefList())**2)
                                   for aLev in levelList if 'ffSmpls' in aLev and 'omnesSmpls' in aLev]).T
            
        # compute covariance matrix from bootstrap samples
        covMat = covBootstrap(theDat)
        cholCov = cholesky(covMat)
        residue = theDat[iCfg]

        return norm(solve_triangular(cholCov, residue, lower=True, check_finite=False))**2 / (residue.shape[0]-nSubtr-1)

    return chiSq

p1Smpls = []

for iCfg in range(nSmpls):
    wProgBar.value = iCfg
    chiSqFunc = configSelector(iCfg)
    
        # either P_1 = c   --or--  P_1 = c + a*s to go with twice-subtracted and thrice-subtracted
    initGuess = [0.3]*(nSubtr-1)
    
    minObj = minimize(chiSqFunc, initGuess)
    p1Smpls.append(minObj.x)
    
    if iCfg == 0:
        print('nLvl =', sum('ffSmpls' in aLev and 'omnesSmpls' in aLev for aLev in levelList))
        print(minObj)
        
p1Smpls = np.array(p1Smpls)

### Plot polynomial fit

In [ ]:
resList = []

for aLev in levelList:
    if 'ffSmpls' not in aLev: continue
    EcmSqOverMpiSqSmpls = np.array(aLev['boxQuant'].getEcmOverMrefList())**2
    EcmSqOverMpiSqErr = errorBootstrap(EcmOverMpiSmpls**2)
    
    remSmpls = np.log(aLev['ffSmpls']/aLev['omnesSmpls'])
    remErr = errorBootstrap(remSmpls)
    resList.append(np.array([EcmSqOverMpiSqSmpls[0], EcmSqOverMpiSqErr[0], EcmSqOverMpiSqErr[1], remSmpls[0], remErr[0], remErr[1]]))

resA = np.array(resList)

    # disp. parametrization data
maxEcmSq = max(aLev['boxQuant'].getEcmOverMrefList()[0] for aLev in levelList)**2
fitX = np.linspace(4.0, maxEcmSq, 200)
fitVal = sum(aPar * fitX**iPar for iPar, aPar in enumerate(p1Smpls[0], start=1))


fig = plt.figure()
ax = fig.add_subplot(111)
ax.errorbar(x=resA[:,0], xerr=resA[:,1:3].T, y=resA[:,3], yerr=resA[:,4:6].T, fmt='k.')
ax.plot(fitX, fitVal, 'k-')

ax.set_xlabel('$E_{cm}^2 / m_\pi^2$')
ax.set_ylabel('$\log (|F_\pi|/\Omega[\delta])$')
fig.show()

### Plot form factor

In [ ]:
resList = []

for aLev in levelList:
    if 'ffSmpls' not in aLev: continue
    EcmOverMpiSmpls = np.array(aLev['boxQuant'].getEcmOverMrefList())
    EcmOverMpiErr = errorBootstrap(EcmOverMpiSmpls)
    
    ffErr = errorBootstrap(aLev['ffSmpls'])
    resList.append(np.array([EcmOverMpiSmpls[0], EcmOverMpiErr[0], EcmOverMpiErr[1], aLev['ffSmpls'][0], ffErr[0], ffErr[1]]))

resA = np.array(resList)

    # parametrization data
maxEcm = max(aLev['boxQuant'].getEcmOverMrefList()[0] for aLev in levelList)
fitX = np.linspace(2.0001, maxEcm, 200)
bwAmpl.mr = mrList[0]
bwAmpl.g = gList[0]
fitVal = np.array([dispFF(nSubtr, bwAmpl.phaseParametrization, lambda s : sum(aPar * s**iPar for iPar, aPar in enumerate(p1Smpls[0], start=1)), aX**2) for aX in fitX])


fig = plt.figure()
ax = fig.add_subplot(111)
ax.errorbar(x=resA[:,0], xerr=resA[:,1:3].T, y=resA[:,3], yerr=resA[:,4:6].T, fmt='k.')
ax.plot(fitX, fitVal, 'k-')

ax.set_xlabel('$E_{cm}/m_\pi$')
ax.set_ylabel('$|F_\pi|$')
fig.show()

### Pion radius in physical units

In [ ]:
latSpac = dM.latParDict['latSpacing']

piRadSmpls = 6*p1Smpls[:,0] / mpiSmpls**2 * latSpac**2
piRadErr = errorBootstrap(piRadSmpls)

print(piRadSmpls[0],piRadErr[0],piRadErr[1])


## Output: ASCII samples

In [ ]:
def writeChimeraSmpls(bootSmpls, fileName):
    with open(fileName, 'w') as fH:
        fH.write(str(len(bootSmpls)) + ' 1 0 0 1\n')

        for aL in bootSmpls:
            fH.write(' 0 ' + str(aL)+'\n')

dM._fetchResults()

    # pion
mpiSmpls = dM.anaDict['pion']['mSmpls'][0]
writeChimeraSmpls(mpiSmpls, 'dat_'+ensName+'/mpi_smpls.dat')

    # BW parameters
writeChimeraSmpls(np.array(mrList), 'dat_'+ensName+'/mr_smpls.dat')
writeChimeraSmpls(np.array(gList), 'dat_'+ensName+'/g_smpls.dat')

    # Omnes parametrization parameters
for iP in range(nSubtr-1):
    writeChimeraSmpls(p1Smpls[:,iP], 'dat_'+ensName+'/dispPoly_nSubtr'+str(nSubtr)+'_p'+str(iP)+'_smpls.dat')

    # p^3 cot, E^*, FF
levelCounter = {}
for aLev in levelList:
    iLev = levelCounter.get(aLev['irrepKey'], 0)

    writeChimeraSmpls(np.array(aLev['boxQuant'].getEcmOverMrefList()), 'dat_'+ensName+'/dSq'+str(aLev['irrepKey'][0])+'_'+aLev['irrepKey'][1]+'_lvl'+str(iLev)+'_Ecm_smpls.dat')
    writeChimeraSmpls(np.array(aLev['boxQuant'].getBoxMatrixElementList(0,0)).real, 'dat_'+ensName+'/dSq'+str(aLev['irrepKey'][0])+'_'+aLev['irrepKey'][1]+'_lvl'+str(iLev)+'_Bmat_smpls.dat')

    if 'cuSmpls' in aLev and 'oaSmpls' in aLev:
        writeChimeraSmpls(aLev['oaSmpls']/aLev['cuSmpls'], 'dat_'+ensName+'/dSq'+str(aLev['irrepKey'][1])+'_'+aLev['irrepKey'][1]+'_lvl'+str(iLev)+'_oacu_smpls.dat')
        
    if 'ffSmpls' in aLev:
        writeChimeraSmpls(aLev['ffSmpls'], 'dat_'+ensName+'/dSq'+str(aLev['irrepKey'][0])+'_'+aLev['irrepKey'][1]+'_lvl'+str(iLev)+'_ff_smpls.dat')
        writeChimeraSmpls(aLev['omnesSmpls'], 'dat_'+ensName+'/dSq'+str(aLev['irrepKey'][0])+'_'+aLev['irrepKey'][1]+'_lvl'+str(iLev)+'_omnes_nSubtr'+str(nSubtr)+'_smpls.dat')

    levelCounter[aLev['irrepKey']] = iLev + 1